<a href="https://colab.research.google.com/github/notingcode/spring2020/blob/master/titanic_data_preprocessing_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analyzing and Preprocessing Data with Matplotlib, Pandas, and NumPy

##Upload Kaggle JSON and Download Data

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions list

In [0]:
!kaggle competitions download -c titanic

##Load and Manipulate Data with Pandas

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [0]:
train_data

In [0]:
test_data

##Analyze Data

Feature(Variable)|Definition|Key
:-|:-|:-
survival|생존|0 = No, 1 = Yes
pclass|티켓 등급|1 = 1st, 2 = 2nd, 3 = 3rd
sex|성별|
Age|나이|
sibsp|함께 탑승중인 형제자매와 배우자#|
parch|함께 탑승중인 부모와 자녀#|
ticket|티켓 번호|
fare|탑승 요금|
cabin|객실 번호|
embarked|탑승한 항구|C = Cherbourg, Q = Queenstown, S = Southampton

In [0]:
%matplotlib inline
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
print(f"{train_data.shape} {test_data.shape}")

In [0]:
train_data.columns

In [0]:
test_data.columns

In [0]:
train_data.dtypes

In [0]:
train_data.isna().sum(axis=0)

In [0]:
test_data.isna().sum(axis=0)

In [0]:
train_data.info()

In [0]:
test_data.info()

In [0]:
noage_train_data = train_data[train_data['Age'].isna()]
havecabin_train_data  = train_data[~train_data['Cabin'].isna()]
knowage_train_data = train_data[~train_data['Age'].isna()]

In [0]:
train_data.describe()

In [0]:
havecabin_train_data.describe()

In [0]:
noage_train_data.describe()

In [0]:
knowage_train_data.describe()

In [0]:
train_data.describe(include=object)

In [0]:
train_data['Embarked'].value_counts(dropna = False)

In [0]:
train_data['Sex'].value_counts(dropna = False)

In [0]:
train_data['Pclass'].value_counts(dropna = False)

In [0]:
age_bins = [i*10 for i in range(9)]

In [0]:
age_group_num = train_data['Age'].value_counts(bins = age_bins, sort = False)
age_group_num

In [0]:
train_data.groupby('Sex')[['Survived']].mean()

In [0]:
train_data.pivot_table('Survived', index='Sex', columns='Pclass')

In [0]:
figs, axes = plt.subplots(2, 2, figsize=(12, 13))

axes[0][0].boxplot(knowage_train_data['Age'])
axes[0][0].set_xlabel("Age")

axes[0][1].boxplot(train_data['Fare'])
axes[0][1].set_xlabel("Fare")

axes[1][0].hist(knowage_train_data['Age'], bins=10)
axes[1][0].set_xlabel("Age")

axes[1][1].hist(train_data['Fare'], bins=10)
axes[1][1].set_xlabel("Fare")

plt.show()

In [0]:
fig, axes = plt.subplots(1, 3, figsize=(25, 9))

sns.countplot(x='Sex', hue='Survived', data=train_data, ax=axes[0])
axes[0].set_title('Survival by sex')
axes[0].set_ylabel('')

sns.countplot(x='Pclass', hue='Survived', data=train_data, ax=axes[1])
axes[1].set_title('Survival by Pclass')
axes[1].set_ylabel('')

sns.countplot(x='Embarked', hue='Survived', data=train_data, ax=axes[2])
axes[2].set_title('Survival by Embarked')
axes[2].set_ylabel('')

plt.show()

In [0]:
plt.pie(train_data.groupby('Sex')[['Survived']].sum(),
        labels=['Female', 'Male'],
        autopct='%1.1f%%')
plt.title("Survival Ratio")

plt.show()

##Preprocessing

In [0]:
train_data = train_data.drop(['PassengerId', 'Ticket'], axis=1)
train_data

In [0]:
sub_example = pd.read_csv('gender_submission.csv')
sub_example.head()

In [0]:
test_data = test_data.drop(['Ticket'], axis=1)

In [0]:
train_data.dtypes

In [0]:
train_data['Family'] = train_data['SibSp'] + train_data['Parch']
test_data['Family'] = test_data['SibSp'] + test_data['Parch']

In [0]:
train_data.groupby('Family')[['Survived']].mean()

In [0]:
# train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
# test_data['Age'].fillna(train_data['Age'].mean(), inplace=True)

In [0]:
def fill_age(df):
  age_avg = train_data['Age'].mean()
  age_std = train_data['Age'].std()
  age_nan_count = df['Age'].isna().sum()
  age_nan_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_nan_count)
  df['Age'][df['Age'].isna()] = age_nan_random_list
  df['Age'] = df['Age'].astype(int)

In [0]:
fill_age(train_data)
fill_age(test_data)

In [0]:
train_data.isna().sum(axis=0)

In [0]:
test_data.isna().sum(axis=0)

In [0]:
train_data["Embarked"].fillna('S', inplace=True)
test_data['Fare'].fillna(train_data['Fare'].median(), inplace=True)

In [0]:
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1}).astype(int)

In [0]:
train_data['Embarked'] = train_data['Embarked'].replace('S', int(0))
train_data['Embarked'] = train_data['Embarked'].replace('C', int(1))
train_data['Embarked'] = train_data['Embarked'].replace('Q', int(2))

embarked_mapping = {'S': 0, 'C': 1, 'Q': 2}
test_data['Embarked'] = test_data['Embarked'].map(embarked_mapping).astype(int)

In [0]:
# train_data['Age'] = train_data['Age'].astype(int)
# train_data['Fare'] = train_data['Fare'].astype(int)

In [0]:
def bin_fare(df):  
  df.loc[df['Fare'] <= 7.91, 'Fare'] = 0
  df.loc[(df['Fare'] > 7.91) & (df['Fare'] <= 14.454), 'Fare'] = 1
  df.loc[(df['Fare'] > 14.454) & (df['Fare'] <= 31), 'Fare'] = 2
  df.loc[df['Fare'] > 31, 'Fare'] =  3

  df['Fare'] = df['Fare'].astype(int)

In [0]:
def bin_age(df):
  df.loc[df['Age'] <= 16, 'Age'] = 0
  df.loc[(df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
  df.loc[(df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
  df.loc[(df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3
  df.loc[df['Age'] > 64, 'Age'] = 4

  df['Age'] = df['Age'].astype(int)

In [0]:
bin_fare(train_data)
bin_fare(test_data)
bin_age(train_data)
bin_age(test_data)

In [0]:
train_data

In [0]:
test_data

In [0]:
type(np.NaN)

In [0]:
train_data['Has_Cabin'] = train_data['Cabin'].apply(lambda x: 0 if type(x) == float else 1)
test_data['Has_Cabin'] = test_data['Cabin'].apply(lambda x: 0 if type(x) == float else 1)

In [0]:
def get_title(name):
  for substring in name.split():
    if '.' in substring:
      return substring[:-1]

train_data['Title'] = train_data['Name'].apply(lambda x: get_title(x))
test_data['Title'] = test_data['Name'].apply(lambda x: get_title(x))

train_data['Title'].value_counts(dropna=False)

In [0]:
test_data['Title'].value_counts(dropna=False)

In [0]:
def map_title(df):
  df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  df['Title'] = df['Title'].replace('Mlle', 'Miss')
  df['Title'] = df['Title'].replace('Ms', 'Miss')
  df['Title'] = df['Title'].replace('Mme', 'Mrs')
  title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
  df['Title'] = df['Title'].map(title_mapping)

In [0]:
map_title(train_data)
map_title(test_data)

In [0]:
train_data.drop(['Name', 'Cabin', 'SibSp', 'Parch'], axis=1, inplace = True)
test_data.drop(['Name', 'Cabin', 'SibSp', 'Parch'], axis=1, inplace = True)

In [0]:
train_data.info()

In [0]:
test_data.info()

In [0]:
train_data

In [0]:
test_data

In [0]:
train_data.to_csv('train_processed.csv')
test_data.to_csv('test_processed.csv')